In [14]:
import os
import torch
import mlflow
import numpy as np
import pandas as pd
from minio import Minio

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
# raw_model, emoPuntu_removed_model, filtered_model
model_list = [
    "s3://mlflow/mlflow/artifacts/9/209c25db460d48eba64907e293cbfb8b/artifacts/electra-test",
    's3://mlflow/mlflow/artifacts/9/c609e84536f14e70be4eee205991c04c/artifacts/electra-test',
    "s3://mlflow/mlflow/artifacts/9/302e3f9eb2714146a62665fe918fd31d/artifacts/electra-test",
]

# raw_data, emoPuntu_removed_data, filtered_data
data_list = [
    "test_phonenumber_data.csv",
    "test_phonenumber_EmoPuntuRemoved.csv",
    "test_phonenumber_PNfiltering.csv",
]

os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://minio-service.kubeflow.svc.cluster.local:9000"

mino_endpoint = os.environ.get(
    "MLFLOW_S3_ENDPOINT_URL", "http://61.80.148.154:30001"
).split("//")[1]

access_key = os.environ["AWS_ACCESS_KEY_ID"]
secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]

client = Minio(
    mino_endpoint,
    access_key=access_key,
    secret_key=secret_key,
    secure=False,
)

device = torch.device("cpu")

/tmp/ipykernel_20724/1625430661.py:26: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.29.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  loaded_model = mlflow.transformers.load_model(
2023/07/05 05:12:37 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

True


In [ ]:
import re
import numpy as np
import pandas as pd

def remove_emoji(s):
    regrex_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+",
        flags=re.UNICODE,
    )
    res = regrex_pattern.sub(r"", s)

    return res


def remove_punctuation(s: str):
    """
    A heuristic approach to check if len of input sentence below 30 after processing it.

    Parameter:
    --------
    s : str
        Input sentence to check if it's a spam or not

    Return:
    --------
    s : str
        String that removed duplicated whitespaces and punctuations.
    """
    regex_list = [
        r"([ㄱ-ㅎㅏ-ㅣ`~!@#$%^&*()_♡\+\=\-,./<'\]'>?;'\\:\|'[''{''}'\"\'])",
        r"[^\w\s]",
    ]
    for r in regex_list:
        s = re.sub(r, "", s)

    return " ".join(s.split())


def phonenumber_filter(s: str):
    """
    A heuristic approach to check if input string contains any phone number.

    Parameter:
    --------
    s : str
        Input sentence that might contain phone number
    Return:
    --------
    res_three : re.Match
    res_double : re.Match
        Part of sentence that may contain possible phone number
    """
    # Remove newline
    s = s.replace("\n", "")

    first_group = [
        r"([영공0])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
    ]

    second_group = [
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
    ]

    third_group = [
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
        r"(.{0,20})",
        r"([영공일이삼사오육륙칠팔구|0-9])",
    ]

    # 1. Check if the processed sentence has phone number pattern
    regex_one = re.compile(
        "".join(first_group) + "".join(second_group) + "".join(third_group)
    )
    regex_two = re.compile("".join(second_group) + "".join(third_group))

    res_three = re.search(regex_one, s)
    res_double = re.search(regex_two, s)

    return (res_three, res_double)


In [15]:
data = pd.read_csv("test_phonenumber_data.csv")

for index, row  in data.iterrows():
    if remove_punctuation(remove_emoji(row["conts"])) == "":
        data.loc[index, "conts"] = np.NaN
    else:
        data.loc[index, "conts"] = remove_punctuation(remove_emoji(row["conts"]))

data = data.dropna()
data.reset_index(inplace=True, drop=True)

# Save 
data.to_csv("test_phonenumber_EmoPuntuRemoved.csv", encoding='utf-8', index=False)

# Read
read_data = pd.read_csv("test_phonenumber_EmoPuntuRemoved.csv")

# Sanity check
assert len(data) == len(read_data), "Sanity Check failed"

for index, row in data.iterrows():
    assert data.loc[index, "conts"] == read_data.loc[index, "conts"]

0    누나들이좋아여 \n 큰거좋아하는누나들만 \n 공일공 육육일오 오팔공칠 연락줘여
Name: 0, dtype: object

precision: 0.9423076923076923 
recall: 0.98 
f1: 0.9607843137254902 
acc: 0.96


In [ ]:
data = pd.read_csv("test_phonenumber_data.csv")

for index, row  in data.iterrows():
    if remove_punctuation(remove_emoji(row["conts"])) == "":
        data.loc[index, "conts"] = np.NaN
    else:
        data.loc[index, "conts"] = remove_punctuation(remove_emoji(row["conts"]))

data = data.dropna()
data.reset_index(inplace=True, drop=True)

def filtering(input):
    three, double = phonenumber_filter(input)

    if not (three or double):
        return input
    elif not three:
        return double.group()
    elif not double:
        return three.group()
    else:
        three_txt, double_txt = three.group(), double.group()
        if len(three_txt) > len(double_txt):
            return three_txt
        else:
            return double_txt

data["conts"] = data["conts"].apply(filtering)

data = data.dropna()
data.reset_index(inplace=True, drop=True)

# Save 
data.to_csv("test_phonenumber_PNfiltering.csv", encoding='utf-8', index=False)

# Read
read_data = pd.read_csv("test_phonenumber_PNfiltering.csv")

# Sanity check
assert len(data) == len(read_data), "Sanity Check failed"

for index, row in data.iterrows():
    assert data.loc[index, "conts"] == read_data.loc[index, "conts"]

In [ ]:
# Load model as a PyFuncModel.
# mlflow.pyfunc.get_model_dependencies(logged_model)

for model, data in zip(model_list, data_list):
    loaded_model = mlflow.transformers.load_model(model_uri=model)
    loaded_data = pd.read_csv(data)
    
    y_pred = [1 if loaded_model.predict(i)[0]["label"]=="LABEL_1" else 0 for i in loaded_data["conts"].values]
    y_true = loaded_data["label"].values

    assert len(y_pred) == len(y_true), "y_pred and y_true have to be the same length"

    for i in range(len(y_pred)):
        if y_pred[i] != y_true[i]:
            print(f"pred:{y_pred[i]}, true:{y_true[i]} \t{loaded_data.loc[i,'conts']}")
        
    
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred, normalize='all')
    
    print(f"\nmodel:{model} \ndata:{loaded_data.loc[1,'conts']}\nprecision: {precision} \nrecall: {recall} \nf1: {f1} \nacc: {acc} \nconfusion_matrix(tn, fp, fn, tp): {confusion.ravel()}")
    print("")